# Création du dataset

Importation des modules

In [ ]:
import os
import pickle

import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

Chargement de la configuration

In [ ]:
import json

with open('config.json', 'r') as f:
   config = json.load(f)

DATA_DIR = config['dataset']['dir']
DATASET_FILE = config['dataset']['file']
DATASET_DEFINITION = config['dataset']['definition']
DATASET_DEFINITION_KEYS = list(DATASET_DEFINITION.keys())

Chargement du modèle 'Hand Pose Detection' permettant de détecter les mains

In [ ]:
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

Fonction permettant de charger l'image et de convertir les couleurs BGR en RGB

In [ ]:
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

Fonction permettant de récupérer les positions X et Y de chaque point de la main.

In [ ]:
def get_hand_data(multi_hand_landmarks):
    data = []
    for hand_landmarks in multi_hand_landmarks:
        for landmark in hand_landmarks.landmark:
           data.append(landmark.x)
           data.append(landmark.y)
    return data

Sauvegarde du dataset qui contient les données des mains dans un fichier

In [ ]:
data = []
labels = []
for dir_ in DATASET_DEFINITION_KEYS:
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        
        img_rgb = load_image(os.path.join(DATA_DIR, dir_, img_path))

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
                    
            hand_data = get_hand_data(results.multi_hand_landmarks)
            
            data.append(hand_data)
            labels.append(dir_)

with open(DATASET_FILE, 'wb') as f:
   pickle.dump({'data': data, 'labels': labels}, f)